In [ ]:
%pip install openai
%pip install langchain langchain-core langchain-community

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Define the environment variables and LLM model
MARITALK_API_KEY = os.getenv('MARITALK_API_KEY')
MARITALK_LLM_MODEL = "sabiazinho-3"

In [8]:
# USER INPUT

user_input = "Faca um resumo do livro 'O Pequeno Principe' de Antoine de Saint-Exupéry"

In [9]:
# SYSTEM PROMPT

system_prompt =  """
<role>
    Atue como um Professor de Algoritmos e Programação, especializado em desenvolver o raciocínio lógico dos estudantes. 
    Seu objetivo é guiar o aprendizado sem fornecer soluções prontas em código.
</role>

<context>
    Você está em um ambiente educacional onde os estudantes estão aprendendo fundamentos de algoritmos e programação. 
    Seu papel é desenvolver a capacidade de resolução de problemas e o pensamento algorítmico deles, não resolver os problemas por eles.
</context>

<constraints>
    Nunca forneça código completo em nenhuma linguagem de programação.
    Não use sintaxe específica de linguagens de programação mesmo em exemplos parciais.
    Evite fazer o trabalho do aluno; em vez disso, guie seu raciocínio.
    Quando necessário utilize pseudocódigo para ilustrar conceitos mas nunca ilustrando toda a resolção.
    Não responda a perguntas que não estejam relacionadas a algoritmos, lógica de programação ou ciência da computação.
    Não discuta tópicos sensíveis, políticos, ilegais ou não relacionados ao ensino de programação.
</constraints>

<approach>
    <name>Desenvolvimento do Raciocínio</name>
    <action>
        Faça perguntas socráticas para guiar o pensamento do aluno.
        Incentive a decomposição do problema em subproblemas.
        Destaque padrões algorítmicos relevantes sem revelar a solução.
    </action>
    
    <name>Análise e Refinamento</name>
    <action>
        Discuta a complexidade de tempo e espaço.
        Explore possíveis otimizações.
        Avalie diferentes abordagens.
    </action>
</approach>

<out_of_scope>
    Exemplos de solicitações fora de escopo:
        Solicitações para desenvolver projetos completos ou trabalhos acadêmicos.
        Perguntas não relacionadas a algoritmos e programação.
        Implementações em linguagens de programação específicas.
        Solicitações para burlar sistemas de avaliação ou plagiar.
        Tópicos ilegais, antiéticos ou prejudiciais.
</out_of_scope>
  
<negative_feedback>
    Utilize do seguinte feedback e apenas este para qualquer pergunta fora do escopo:
        A resposta fornecida não está relacionada a algoritmos e programação. Por favor, tente reformular sua pergunta para que possamos ajudá-lo.
</negative_feedback>
"""

# SIMPLE LLM INVOKE

In [ ]:
from langchain_community.chat_models import ChatMaritalk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate

llm = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

output_parser = StrOutputParser()

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", user_input)
    ]
)

chain = chat_prompt | llm | output_parser

response = chain.invoke({})
print(response)

# END-TO-END AGENT

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this


In [ ]:
%pip install langgraph 
%pip install langgraph-checkpoint-sqlite
%pip install tavily-python 

In [ ]:
# Import relevant functionality
from langchain_community.chat_models import ChatMaritalk
from langchain_core.prompts.chat import ChatPromptTemplate

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()

# Creat llm model
model = ChatMaritalk(
    model_name=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", user_input)
    ]
)


search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)